# Tools to work with tables extracted from csi softwares ETABS,SAFE,SAP2000

In [53]:

from pathlib import Path
import pandas as pd
import openpyxl
Path.cwd().resolve()

WindowsPath('C:/Users/m.ali/OneDrive - Hesco/2025/_automation/csi')

In [54]:
# pip install -e .
# pip uninstall mat
import mat
mat.say_hi()

Hello World 2025-03-25 02:44:42


'Hello World 2025-03-25 02:44:42'

In [55]:
folder_in = Path('./sample_data')
folder_out = Path('./sample_output')
folder_in.resolve()
sheet_names = {
    'joints' : 'Objects and Elements - Joints',
    'joint_forces' : 'Element Joint Forces - Shells',
    'areas' : 'Objects and Elements - Areas',
    'area_stresses' : 'Element Stresses - Area Shells',
    'area_forces' : 'Element Forces - Area Shells',
}
sheet_names
xlfile = mat.find_files(folder_in,recursive=True)[0]

In [56]:
joint_df = pd.read_excel(
    xlfile,
    sheet_name = sheet_names['joints'],
    skiprows=[0,2],
)
joint_df = joint_df[['Story','Element Name','Global X','Global Y','Global Z']]
joint_df = joint_df.rename(columns={
    'Element Name':'name',
    'Story' : 'story',
    'Global X' : 'x',
    'Global Y' : 'y',
    'Global Z' : 'z',
})
joint_df.loc[joint_df.story == 'BASE']

,story,name,x,y,z
3000,BASE,24,0.0,10000.0,0
3001,BASE,36,10000.0,10000.0,0
3002,BASE,42,10000.0,0.0,0
3003,BASE,48,0.0,0.0,0
3004,BASE,54,5000.0,5000.0,0
...,...,...,...,...,...
3832,BASE,~3833,10725.2,293.7,0
3833,BASE,~3834,10754.4,10.6,0
3834,BASE,~3835,10554.1,19.4,0
3835,BASE,~3836,10724.8,-259.2,0


In [57]:
area_df = pd.read_excel(
    xlfile,
    sheet_name=sheet_names['areas'],
    skiprows=[0,2]
)
area_df = area_df.drop(columns=['Object Type','Object Label','Object Name'])
area_df = area_df.rename(columns={
    'Story' : 'story',
    'Element Name' : 'name',
    'Elm Jt1' : 'jt1',
    'Elm Jt2' : 'jt2',
    'Elm Jt3' : 'jt3',
    'Elm Jt4' : 'jt4',
})
area_df.loc[area_df.story == 'BASE']

,story,name,jt1,jt2,jt3,jt4
2940,BASE,1-1,36,~3010,~3011,~3012
2941,BASE,1-2,~3013,~3014,~3011,~3010
2942,BASE,1-3,~3014,~3015,~3016,~3011
2943,BASE,1-4,~3015,~3017,~3018,~3016
2944,BASE,1-5,~3017,~3019,~3020,~3018
...,...,...,...,...,...,...
3752,BASE,1-813,~3837,~3831,~3834,~3836
3753,BASE,1-814,~3834,~3833,~3835,NaN
3754,BASE,1-815,~3834,~3835,~3836,NaN
3755,BASE,1-816,~3201,~3202,~3211,NaN


In [58]:
stress_df = pd.read_excel(
    xlfile,
    sheet_name = sheet_names['area_stresses'],
    skiprows = [0,2],
    usecols = "A,E:G,K:Y"
)
stress_df
stress_df = stress_df.loc[(stress_df['Story'] == 'BASE') & (stress_df['Output Case'] == '1.4D+1.7L')]


In [59]:
stress_df = stress_df.merge(joint_df, how='left', left_on=['Joint'], right_on=['name'])
stress_df

,Story,Shell Element,Joint,Output Case,S11 Top,S22 Top,S12 Top,SMax Top,SMin Top,SVM Top,...,SMin Bottom,SVM Bottom,S13 Average,S23 Average,SMaxV Average,story,name,x,y,z
0,BASE,1-1,36,1.4D+1.7L,-7.94,-6.30,3.17,-3.85,-10.40,9.10,...,3.88,9.10,-0.75,2.20,2.32,BASE,36,10000.0,10000.0,0
1,BASE,1-1,~3010,1.4D+1.7L,-3.33,-1.96,2.53,-0.02,-5.27,5.26,...,0.05,5.24,0.51,1.09,1.20,BASE,~3010,10334.8,10321.5,0
2,BASE,1-1,~3011,1.4D+1.7L,-3.68,-1.17,2.09,0.01,-4.86,4.87,...,-0.02,4.86,0.53,0.80,0.96,BASE,~3011,9976.3,10550.1,0
3,BASE,1-1,~3012,1.4D+1.7L,-4.62,-2.67,0.54,-2.54,-4.75,4.12,...,2.54,4.11,-0.83,1.95,2.12,BASE,~3012,9710.6,10220.8,0
4,BASE,1-2,~3013,1.4D+1.7L,-1.52,-0.99,1.11,-0.12,-2.39,2.34,...,0.12,2.33,0.24,0.18,0.30,BASE,~3013,10708.7,10705.4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3205,BASE,1-816,~3202,1.4D+1.7L,2.39,2.02,1.91,4.12,0.28,3.99,...,-4.12,3.99,-0.01,0.06,0.06,BASE,~3202,6337.4,9497.0,0
3206,BASE,1-816,~3211,1.4D+1.7L,2.19,2.33,1.46,3.72,0.79,3.40,...,-3.73,3.40,-0.01,0.06,0.06,BASE,~3211,5988.3,9360.6,0
3207,BASE,1-817,~3584,1.4D+1.7L,1.50,2.76,-1.72,3.96,0.30,3.82,...,-3.96,3.82,-0.02,0.06,0.06,BASE,~3584,9907.5,3638.2,0
3208,BASE,1-817,~3585,1.4D+1.7L,1.14,3.03,-2.11,4.40,-0.22,4.52,...,-4.40,4.52,-0.02,0.06,0.06,BASE,~3585,10150.4,3079.1,0


In [63]:
stress_df_grouped = stress_df.groupby('Shell Element')[['Joint', 'S13 Average','S23 Average','SMaxV Average','x','y','z']].agg(lambda x: tuple(x)).reset_index()
stress_df_grouped = stress_df_grouped.rename(columns={
    'S13 Average' : 's13',
    'S23 Average' : 's23',
    'SMaxV Average' : 's_max_v',
    'Joint': 'joint',
    'Shell Element' : 'shell',
})
stress_df_grouped.to_excel(folder_out / 'shear_summary.xlsx', index = False)
stress_df_grouped


,shell,joint,s13,s23,s_max_v,x,y,z
0,1-1,"(36, ~3010, ~3011, ~3012)","(-0.75, 0.51, 0.53, -0.83)","(2.2, 1.09, 0.8, 1.95)","(2.32, 1.2, 0.96, 2.12)","(10000.0, 10334.8, 9976.3, 9710.6)","(10000.0, 10321.5, 10550.1, 10220.8)","(0, 0, 0, 0)"
1,1-10,"(~3027, ~3029, ~3030, ~3028)","(-0.05, -0.05, -0.04, -0.04)","(-0.11, -0.15, -0.15, -0.11)","(0.12, 0.16, 0.16, 0.11)","(7000.0, 6500.0, 6508.6, 7005.3)","(11000.0, 11000.0, 10649.9, 10637.0)","(0, 0, 0, 0)"
2,1-100,"(~3149, ~3146, ~3145, ~3139)","(0.32, 0.21, 0.28, 0.35)","(-0.14, -0.01, -0.02, -0.11)","(0.34, 0.21, 0.28, 0.36)","(1520.3, 1969.3, 1786.9, 1091.6)","(9562.4, 9749.6, 10246.1, 10019.0)","(0, 0, 0, 0)"
3,1-101,"(~3139, ~3065, ~3067, ~3149)","(0.47, 0.51, 0.32, 0.32)","(0.08, 0.06, -0.26, -0.17)","(0.47, 0.51, 0.41, 0.36)","(1091.6, 897.6, 1229.9, 1520.3)","(10019.0, 9617.6, 9284.8, 9562.4)","(0, 0, 0, 0)"
4,1-102,"(~3079, ~3081, ~3150, ~3151)","(-0.15, -0.08, -0.1, -0.16)","(0.4, 0.45, 0.47, 0.41)","(0.42, 0.46, 0.47, 0.44)","(3387.4, 3724.1, 4098.0, 3763.1)","(7297.8, 6949.2, 7255.0, 7614.9)","(0, 0, 0, 0)"
...,...,...,...,...,...,...,...,...
812,1-95,"(~3048, ~3145, ~3140, ~3046)","(0.16, 0.12, 0.13, 0.17)","(-0.07, -0.08, -0.09, -0.08)","(0.17, 0.15, 0.16, 0.19)","(1943.1, 1786.9, 2374.8, 2436.7)","(10622.1, 10246.1, 10227.7, 10620.7)","(0, 0, 0, 0)"
813,1-96,"(~3146, ~3147, ~3144)","(0.2, 0.2, 0.2)","(-0.02, -0.02, -0.02)","(0.2, 0.2, 0.2)","(1969.3, 2146.7, 2439.8)","(9749.6, 9440.2, 9751.5)","(0, 0, 0)"
814,1-97,"(~3146, ~3144, ~3140, ~3145)","(0.16, 0.15, 0.17, 0.18)","(-0.01, -0.05, -0.07, -0.03)","(0.16, 0.16, 0.19, 0.19)","(1969.3, 2439.8, 2374.8, 1786.9)","(9749.6, 9751.5, 10227.7, 10246.1)","(0, 0, 0, 0)"
815,1-98,"(~3148, ~3147, ~3146, ~3149)","(0.22, 0.2, 0.17, 0.2)","(-0.03, -0.06, -0.11, -0.08)","(0.22, 0.21, 0.2, 0.21)","(1790.4, 2146.7, 1969.3, 1520.3)","(9196.3, 9440.2, 9749.6, 9562.4)","(0, 0, 0, 0)"


In [69]:
stress_df_grouped['s_max_v']

0       (2.32, 1.2, 0.96, 2.12)
1      (0.12, 0.16, 0.16, 0.11)
2      (0.34, 0.21, 0.28, 0.36)
3      (0.47, 0.51, 0.41, 0.36)
4      (0.42, 0.46, 0.47, 0.44)
                 ...           
812    (0.17, 0.15, 0.16, 0.19)
813             (0.2, 0.2, 0.2)
814    (0.16, 0.16, 0.19, 0.19)
815     (0.22, 0.21, 0.2, 0.21)
816    (0.23, 0.23, 0.24, 0.23)
Name: s_max_v, Length: 817, dtype: object

In [61]:
for key,value in stress_df_grouped:
    print(key)
    print('-' * 10)
    print(value)

ValueError: too many values to unpack (expected 2)